In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from quantfreedom.indicators.tv_indicators import rsi_tv, bb_tv, macd_tv

sns.set_style('whitegrid')
idx = pd.IndexSlice

In [ ]:
DATA_STORE = '../data/assets.h5'

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    data: pd.DataFrame = (
        store["quandl/wiki/prices"]
        .loc[idx["2007":"2010", "AAPL"], ["adj_open", "adj_high", "adj_low", "adj_close", "adj_volume"]]
        .unstack("ticker")
        .swaplevel(axis=1)
        .loc[:, "AAPL"]
        .rename(columns=lambda x: x.replace("adj_", ""))
    )

In [ ]:
data

In [ ]:
closing_prices = data["close"].to_numpy()

In [ ]:
mid, up, low = bb_tv(source=closing_prices, length=21, multi=2)

In [ ]:
rsi = rsi_tv(source=closing_prices, length=14)

In [ ]:
macdhist, macd, macdsignal = macd_tv(source=closing_prices, fast_length=12, slow_length=26, signal_smoothing=9)

In [ ]:
macd_data = pd.DataFrame({"AAPL": data.close, "MACD": macd, "MACD Signal": macdsignal, "MACD History": macdhist})

fig, axes = plt.subplots(nrows=2, figsize=(15, 8))
macd_data.AAPL.plot(ax=axes[0])
macd_data.drop("AAPL", axis=1).plot(ax=axes[1])
fig.tight_layout()
sns.despine()

In [ ]:
data = pd.DataFrame({'AAPL': data.close, 'BB Up': up, 'BB Mid': mid, 'BB down': low, 'RSI': rsi, 'MACD': macd})

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(15, 10), sharex=True)
data.drop(["RSI", "MACD"], axis=1).plot(ax=axes[0], lw=1, title="Bollinger Bands")
data["RSI"].plot(ax=axes[1], lw=1, title="Relative Strength Index")
axes[1].axhline(70, lw=1, ls="--", c="k")
axes[1].axhline(30, lw=1, ls="--", c="k")
data.MACD.plot(ax=axes[2], lw=1, title="Moving Average Convergence/Divergence", rot=0)
axes[2].set_xlabel("")
fig.tight_layout()
sns.despine()